In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re
from collections import Counter
import liwc
import math
import json
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import arabic_reviews

In [2]:
data = pd.read_json("EnglishBookReviews.json")

In [3]:
data_conf = data[:1480]
data_conf["helpfulness"] = data['n_votes'][:1480].map(lambda x: "helpful" if x > 0 else "unhelpful")
classes = data_conf["helpfulness"]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Features

In [4]:
data_conf.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,helpfulness
0,12889ca3ea33be08b182e1bab60e8b55,13087982,986067c40edc87e152e4ec317c4b56c3,5,I don't even know how to review this. I feel l...,Mon Mar 03 14:49:09 -0800 2014,Tue Sep 19 20:42:44 -0700 2017,Thu Jan 19 00:00:00 -0800 2017,Sun Sep 20 00:00:00 -0700 2015,3,0,helpful
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,,,1,0,helpful
2,6ba3427be7a706f83d079e8ec902a48d,14061957,d6f80a1b3eaee0b640d5a99e1e7c601b,5,"""Maybe love was superstition, a prayer we said...",Mon Sep 28 22:00:23 -0700 2015,Sun Oct 18 19:10:31 -0700 2015,Mon Sep 28 00:00:00 -0700 2015,Mon Sep 28 00:00:00 -0700 2015,1,0,helpful
3,424956bc779a611313ff39813ed4ca14,35181314,ec7d733d477cee5cd377eea238fea747,5,Review to come,Tue Apr 04 10:56:56 -0700 2017,Fri Oct 20 11:11:43 -0700 2017,Fri Oct 20 14:12:21 -0700 2017,Thu Sep 28 23:39:33 -0700 2017,1,0,helpful
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1,helpful


In [5]:
struct = arabic_reviews.struct_extract(data_conf["review_text"])
galc = arabic_reviews.galc_extract(data_conf["review_text"])
inq = arabic_reviews.inq_extract(data_conf["review_text"])
liwc = arabic_reviews.liwc_extract(data_conf["review_text"])

In [7]:
arabic_reviews.ex_with_best_k(galc,classes,100)

k = ----------  1
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
SVM
Confusion Matrix : 
[[11 57]
 [ 7 73]]
Acc:  0.568  Prec:  0.562  Recall:  0.912  F1: 0.695
Decision Tree
Confusion Matrix : 
[[26 42]
 [16 64]]
Acc:  0.608  Prec:  0.604  Recall:  0.8  F1: 0.688
Random Forest
Confusion Matrix : 
[[26 42]
 [16 64]]
Acc:  0.608  Prec:  0.604  Recall:  0.8  F1: 0.688
NN
Confusion Matrix : 
[[18 50]
 [ 9 71]]
Acc:  0.601  Prec:  0.587  Recall:  0.888  F1: 0.706
k = ----------  2
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
SVM
Confusion Matrix : 
[[13 55]
 [ 7 73]]
Acc:  0.581  Prec:  0.57  Recall:  0.912  F1: 0.702
Decision Tree
Confusion Matrix : 
[[25 43]
 [15 65]]
Acc:  0.608  Prec:  0.602  Recall:  0.812  F1: 

Random Forest
Confusion Matrix : 
[[ 9 59]
 [ 8 72]]
Acc:  0.547  Prec:  0.55  Recall:  0.9  F1: 0.682
NN
Confusion Matrix : 
[[24 44]
 [14 66]]
Acc:  0.608  Prec:  0.6  Recall:  0.825  F1: 0.695
k = ----------  15
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
SVM
Confusion Matrix : 
[[14 54]
 [ 8 72]]
Acc:  0.581  Prec:  0.571  Recall:  0.9  F1: 0.699
Decision Tree
Confusion Matrix : 
[[25 43]
 [16 64]]
Acc:  0.601  Prec:  0.598  Recall:  0.8  F1: 0.684
Random Forest
Confusion Matrix : 
[[12 56]
 [ 7 73]]
Acc:  0.574  Prec:  0.566  Recall:  0.912  F1: 0.699
NN
Confusion Matrix : 
[[21 47]
 [16 64]]
Acc:  0.574  Prec:  0.577  Recall:  0.8  F1: 0.67
k = ----------  16
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 1 79]]
Acc:  0.534  Prec:  0.537  Recall:  0.988  F1: 0.69

Acc:  0.581  Prec:  0.59  Recall:  0.738  F1: 0.656
Random Forest
Confusion Matrix : 
[[ 9 59]
 [ 6 74]]
Acc:  0.561  Prec:  0.556  Recall:  0.925  F1: 0.695
NN
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
k = ----------  29
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 3 77]]
Acc:  0.52  Prec:  0.531  Recall:  0.962  F1: 0.684
SVM
Confusion Matrix : 
[[17 51]
 [ 8 72]]
Acc:  0.601  Prec:  0.585  Recall:  0.9  F1: 0.709
Decision Tree
Confusion Matrix : 
[[23 45]
 [19 61]]
Acc:  0.568  Prec:  0.575  Recall:  0.762  F1: 0.656
Random Forest
Confusion Matrix : 
[[14 54]
 [ 8 72]]
Acc:  0.581  Prec:  0.571  Recall:  0.9  F1: 0.699
NN
Confusion Matrix : 
[[23 45]
 [17 63]]
Acc:  0.581  Prec:  0.583  Recall:  0.788  F1: 0.67
k = ----------  30
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 

ValueError: k should be >=0, <= n_features = 38; got 39. Use k='all' to return all features.

In [8]:
arabic_reviews.ex_with_best_k(struct,classes,7)

k = ----------  1
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
SVM
Confusion Matrix : 
[[16 52]
 [ 3 77]]
Acc:  0.628  Prec:  0.597  Recall:  0.962  F1: 0.737
Decision Tree
Confusion Matrix : 
[[32 36]
 [26 54]]
Acc:  0.581  Prec:  0.6  Recall:  0.675  F1: 0.635
Random Forest
Confusion Matrix : 
[[32 36]
 [26 54]]
Acc:  0.581  Prec:  0.6  Recall:  0.675  F1: 0.635
NN
Confusion Matrix : 
[[21 47]
 [ 5 75]]
Acc:  0.649  Prec:  0.615  Recall:  0.938  F1: 0.743
k = ----------  2
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
SVM
Confusion Matrix : 
[[23 45]
 [ 8 72]]
Acc:  0.642  Prec:  0.615  Recall:  0.9  F1: 0.731
Decision Tree
Confusion Matrix : 
[[28 40]
 [27 53]]
Acc:  0.547  Prec:  0.57  Recall:  0.662  F1: 0.

In [17]:
arabic_reviews.ex_with_best_k(inq[1:],classes,100)

k = ----------  1
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
SVM
Confusion Matrix : 
[[14 54]
 [ 3 77]]
Acc:  0.615  Prec:  0.588  Recall:  0.962  F1: 0.73
Decision Tree
Confusion Matrix : 
[[30 38]
 [24 56]]
Acc:  0.581  Prec:  0.596  Recall:  0.7  F1: 0.644
Random Forest
Confusion Matrix : 
[[30 38]
 [24 56]]
Acc:  0.581  Prec:  0.596  Recall:  0.7  F1: 0.644
NN
Confusion Matrix : 
[[30 38]
 [24 56]]
Acc:  0.581  Prec:  0.596  Recall:  0.7  F1: 0.644
k = ----------  2
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
SVM
Confusion Matrix : 
[[19 49]
 [ 8 72]]
Acc:  0.615  Prec:  0.595  Recall:  0.9  F1: 0.716
Decision Tree
Confusion Matrix : 
[[22 46]
 [16 64]]
Acc:  0.581  Prec:  0.582  Recall:  0.8  F1: 0.674


Random Forest
Confusion Matrix : 
[[33 35]
 [20 60]]
Acc:  0.628  Prec:  0.632  Recall:  0.75  F1: 0.686
NN
Confusion Matrix : 
[[26 42]
 [21 59]]
Acc:  0.574  Prec:  0.584  Recall:  0.738  F1: 0.652
k = ----------  15
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 2 66]
 [ 0 80]]
Acc:  0.554  Prec:  0.548  Recall:  1.0  F1: 0.708
SVM
Confusion Matrix : 
[[24 44]
 [11 69]]
Acc:  0.628  Prec:  0.611  Recall:  0.862  F1: 0.715
Decision Tree
Confusion Matrix : 
[[38 30]
 [29 51]]
Acc:  0.601  Prec:  0.63  Recall:  0.638  F1: 0.634
Random Forest
Confusion Matrix : 
[[34 34]
 [22 58]]
Acc:  0.622  Prec:  0.63  Recall:  0.725  F1: 0.674
NN
Confusion Matrix : 
[[28 40]
 [23 57]]
Acc:  0.574  Prec:  0.588  Recall:  0.712  F1: 0.644
k = ----------  16
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 3 65]
 [ 0 80]]
Acc:  0.561  Prec:  0.552  Recall:  1.0  F

Random Forest
Confusion Matrix : 
[[36 32]
 [24 56]]
Acc:  0.622  Prec:  0.636  Recall:  0.7  F1: 0.667
NN
Confusion Matrix : 
[[26 42]
 [30 50]]
Acc:  0.514  Prec:  0.543  Recall:  0.625  F1: 0.581
k = ----------  29
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 6 62]
 [ 0 80]]
Acc:  0.581  Prec:  0.563  Recall:  1.0  F1: 0.721
SVM
Confusion Matrix : 
[[24 44]
 [13 67]]
Acc:  0.615  Prec:  0.604  Recall:  0.838  F1: 0.702
Decision Tree
Confusion Matrix : 
[[29 39]
 [36 44]]
Acc:  0.493  Prec:  0.53  Recall:  0.55  F1: 0.54
Random Forest
Confusion Matrix : 
[[35 33]
 [21 59]]
Acc:  0.635  Prec:  0.641  Recall:  0.738  F1: 0.686
NN
Confusion Matrix : 
[[28 40]
 [32 48]]
Acc:  0.514  Prec:  0.545  Recall:  0.6  F1: 0.571
k = ----------  30
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 6 62]
 [ 0 80]]
Acc:  0.581  Prec:  0.563  Recall:  1.0  F1: 0

Random Forest
Confusion Matrix : 
[[36 32]
 [26 54]]
Acc:  0.608  Prec:  0.628  Recall:  0.675  F1: 0.651
NN
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
k = ----------  43
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 9 59]
 [ 0 80]]
Acc:  0.601  Prec:  0.576  Recall:  1.0  F1: 0.731
SVM
Confusion Matrix : 
[[24 44]
 [14 66]]
Acc:  0.608  Prec:  0.6  Recall:  0.825  F1: 0.695
Decision Tree
Confusion Matrix : 
[[35 33]
 [37 43]]
Acc:  0.527  Prec:  0.566  Recall:  0.538  F1: 0.551
Random Forest
Confusion Matrix : 
[[33 35]
 [24 56]]
Acc:  0.601  Prec:  0.615  Recall:  0.7  F1: 0.655
NN
Confusion Matrix : 
[[22 46]
 [25 55]]
Acc:  0.52  Prec:  0.545  Recall:  0.688  F1: 0.608
k = ----------  44
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[11 57]
 [ 0 80]]
Acc:  0.615  Prec:  0.584  Recall:  1.0  F1: 0

Confusion Matrix : 
[[35 33]
 [31 49]]
Acc:  0.568  Prec:  0.598  Recall:  0.612  F1: 0.605
Random Forest
Confusion Matrix : 
[[35 33]
 [23 57]]
Acc:  0.622  Prec:  0.633  Recall:  0.712  F1: 0.671
NN
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
k = ----------  57
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[10 58]
 [ 0 80]]
Acc:  0.608  Prec:  0.58  Recall:  1.0  F1: 0.734
SVM
Confusion Matrix : 
[[24 44]
 [14 66]]
Acc:  0.608  Prec:  0.6  Recall:  0.825  F1: 0.695
Decision Tree
Confusion Matrix : 
[[33 35]
 [40 40]]
Acc:  0.493  Prec:  0.533  Recall:  0.5  F1: 0.516
Random Forest
Confusion Matrix : 
[[35 33]
 [23 57]]
Acc:  0.622  Prec:  0.633  Recall:  0.712  F1: 0.671
NN
Confusion Matrix : 
[[29 39]
 [30 50]]
Acc:  0.534  Prec:  0.562  Recall:  0.625  F1: 0.592
k = ----------  58
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1

Acc:  0.581  Prec:  0.605  Recall:  0.65  F1: 0.627
Random Forest
Confusion Matrix : 
[[33 35]
 [22 58]]
Acc:  0.615  Prec:  0.624  Recall:  0.725  F1: 0.671
NN
Confusion Matrix : 
[[35 33]
 [28 52]]
Acc:  0.588  Prec:  0.612  Recall:  0.65  F1: 0.63
k = ----------  71
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[11 57]
 [ 5 75]]
Acc:  0.581  Prec:  0.568  Recall:  0.938  F1: 0.708
SVM
Confusion Matrix : 
[[25 43]
 [16 64]]
Acc:  0.601  Prec:  0.598  Recall:  0.8  F1: 0.684
Decision Tree
Confusion Matrix : 
[[36 32]
 [30 50]]
Acc:  0.581  Prec:  0.61  Recall:  0.625  F1: 0.617
Random Forest
Confusion Matrix : 
[[33 35]
 [22 58]]
Acc:  0.615  Prec:  0.624  Recall:  0.725  F1: 0.671
NN
Confusion Matrix : 
[[30 38]
 [35 45]]
Acc:  0.507  Prec:  0.542  Recall:  0.562  F1: 0.552
k = ----------  72
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[10 58]


Decision Tree
Confusion Matrix : 
[[33 35]
 [35 45]]
Acc:  0.527  Prec:  0.562  Recall:  0.562  F1: 0.562
Random Forest
Confusion Matrix : 
[[36 32]
 [24 56]]
Acc:  0.622  Prec:  0.636  Recall:  0.7  F1: 0.667
NN
Confusion Matrix : 
[[32 36]
 [32 48]]
Acc:  0.541  Prec:  0.571  Recall:  0.6  F1: 0.585
k = ----------  85
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[13 55]
 [ 7 73]]
Acc:  0.581  Prec:  0.57  Recall:  0.912  F1: 0.702
SVM
Confusion Matrix : 
[[25 43]
 [17 63]]
Acc:  0.595  Prec:  0.594  Recall:  0.788  F1: 0.677
Decision Tree
Confusion Matrix : 
[[32 36]
 [37 43]]
Acc:  0.507  Prec:  0.544  Recall:  0.538  F1: 0.541
Random Forest
Confusion Matrix : 
[[34 34]
 [24 56]]
Acc:  0.608  Prec:  0.622  Recall:  0.7  F1: 0.659
NN
Confusion Matrix : 
[[30 38]
 [30 50]]
Acc:  0.541  Prec:  0.568  Recall:  0.625  F1: 0.595
k = ----------  86
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers 

Confusion Matrix : 
[[27 41]
 [37 43]]
Acc:  0.473  Prec:  0.512  Recall:  0.538  F1: 0.524
Random Forest
Confusion Matrix : 
[[33 35]
 [23 57]]
Acc:  0.608  Prec:  0.62  Recall:  0.712  F1: 0.663
NN
Confusion Matrix : 
[[32 36]
 [30 50]]
Acc:  0.554  Prec:  0.581  Recall:  0.625  F1: 0.602
k = ----------  99
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[14 54]
 [ 6 74]]
Acc:  0.595  Prec:  0.578  Recall:  0.925  F1: 0.712
SVM
Confusion Matrix : 
[[26 42]
 [18 62]]
Acc:  0.595  Prec:  0.596  Recall:  0.775  F1: 0.674
Decision Tree
Confusion Matrix : 
[[29 39]
 [36 44]]
Acc:  0.493  Prec:  0.53  Recall:  0.55  F1: 0.54
Random Forest
Confusion Matrix : 
[[33 35]
 [23 57]]
Acc:  0.608  Prec:  0.62  Recall:  0.712  F1: 0.663
NN
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
0.6486486486486487


In [18]:
arabic_reviews.ex_with_best_k(liwc,classes,100)

k = ----------  1
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
SVM
Confusion Matrix : 
[[16 52]
 [ 5 75]]
Acc:  0.615  Prec:  0.591  Recall:  0.938  F1: 0.725
Decision Tree
Confusion Matrix : 
[[28 40]
 [16 64]]
Acc:  0.622  Prec:  0.615  Recall:  0.8  F1: 0.696
Random Forest
Confusion Matrix : 
[[29 39]
 [16 64]]
Acc:  0.628  Prec:  0.621  Recall:  0.8  F1: 0.699
NN
Confusion Matrix : 
[[23 45]
 [ 8 72]]
Acc:  0.642  Prec:  0.615  Recall:  0.9  F1: 0.731
k = ----------  2
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
SVM
Confusion Matrix : 
[[22 46]
 [ 8 72]]
Acc:  0.635  Prec:  0.61  Recall:  0.9  F1: 0.727
Decision Tree
Confusion Matrix : 
[[29 39]
 [22 58]]
Acc:  0.588  Prec:  0.598  Recall:  0.725  F1: 0.65

Random Forest
Confusion Matrix : 
[[33 35]
 [23 57]]
Acc:  0.608  Prec:  0.62  Recall:  0.712  F1: 0.663
NN
Confusion Matrix : 
[[29 39]
 [22 58]]
Acc:  0.588  Prec:  0.598  Recall:  0.725  F1: 0.655
k = ----------  15
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 1 67]
 [ 0 80]]
Acc:  0.547  Prec:  0.544  Recall:  1.0  F1: 0.705
SVM
Confusion Matrix : 
[[26 42]
 [10 70]]
Acc:  0.649  Prec:  0.625  Recall:  0.875  F1: 0.729
Decision Tree
Confusion Matrix : 
[[33 35]
 [44 36]]
Acc:  0.466  Prec:  0.507  Recall:  0.45  F1: 0.477
Random Forest
Confusion Matrix : 
[[33 35]
 [25 55]]
Acc:  0.595  Prec:  0.611  Recall:  0.688  F1: 0.647
NN
Confusion Matrix : 
[[31 37]
 [14 66]]
Acc:  0.655  Prec:  0.641  Recall:  0.825  F1: 0.721
k = ----------  16
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 1 67]
 [ 0 80]]
Acc:  0.547  Prec:  0.544  Recall:  1.0  

Random Forest
Confusion Matrix : 
[[36 32]
 [25 55]]
Acc:  0.615  Prec:  0.632  Recall:  0.688  F1: 0.659
NN
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
k = ----------  29
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 3 65]
 [ 0 80]]
Acc:  0.561  Prec:  0.552  Recall:  1.0  F1: 0.711
SVM
Confusion Matrix : 
[[26 42]
 [12 68]]
Acc:  0.635  Prec:  0.618  Recall:  0.85  F1: 0.716
Decision Tree
Confusion Matrix : 
[[31 37]
 [33 47]]
Acc:  0.527  Prec:  0.56  Recall:  0.588  F1: 0.573
Random Forest
Confusion Matrix : 
[[34 34]
 [24 56]]
Acc:  0.608  Prec:  0.622  Recall:  0.7  F1: 0.659
NN
Confusion Matrix : 
[[37 31]
 [22 58]]
Acc:  0.642  Prec:  0.652  Recall:  0.725  F1: 0.686
k = ----------  30
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 3 65]
 [ 0 80]]
Acc:  0.561  Prec:  0.552  Recall:  1.0  F1: 

Random Forest
Confusion Matrix : 
[[34 34]
 [25 55]]
Acc:  0.601  Prec:  0.618  Recall:  0.688  F1: 0.651
NN
Confusion Matrix : 
[[32 36]
 [25 55]]
Acc:  0.588  Prec:  0.604  Recall:  0.688  F1: 0.643
k = ----------  43
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 3 65]
 [ 1 79]]
Acc:  0.554  Prec:  0.549  Recall:  0.988  F1: 0.705
SVM
Confusion Matrix : 
[[25 43]
 [14 66]]
Acc:  0.615  Prec:  0.606  Recall:  0.825  F1: 0.698
Decision Tree
Confusion Matrix : 
[[35 33]
 [39 41]]
Acc:  0.514  Prec:  0.554  Recall:  0.512  F1: 0.532
Random Forest
Confusion Matrix : 
[[34 34]
 [25 55]]
Acc:  0.601  Prec:  0.618  Recall:  0.688  F1: 0.651
NN
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
k = ----------  44
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 3 65]
 [ 1 79]]
Acc:  0.554  Prec:  0.549  Recall:  0.9

Random Forest
Confusion Matrix : 
[[34 34]
 [25 55]]
Acc:  0.601  Prec:  0.618  Recall:  0.688  F1: 0.651
NN
Confusion Matrix : 
[[ 0 68]
 [ 0 80]]
Acc:  0.541  Prec:  0.541  Recall:  1.0  F1: 0.702
k = ----------  57
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 4 64]
 [ 1 79]]
Acc:  0.561  Prec:  0.552  Recall:  0.988  F1: 0.709
SVM
Confusion Matrix : 
[[26 42]
 [11 69]]
Acc:  0.642  Prec:  0.622  Recall:  0.862  F1: 0.723
Decision Tree
Confusion Matrix : 
[[37 31]
 [40 40]]
Acc:  0.52  Prec:  0.563  Recall:  0.5  F1: 0.53
Random Forest
Confusion Matrix : 
[[34 34]
 [25 55]]
Acc:  0.601  Prec:  0.618  Recall:  0.688  F1: 0.651
NN
Confusion Matrix : 
[[34 34]
 [32 48]]
Acc:  0.554  Prec:  0.585  Recall:  0.6  F1: 0.593
k = ----------  58
Classes found :  ['helpful' 'unhelpful']

Classes converted to integers : [0 0 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 4 64]
 [ 1 79]]
Acc:  0.561  Prec:  0.552  Recall:  0.988  F1

ValueError: k should be >=0, <= n_features = 64; got 65. Use k='all' to return all features.